In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("data/output_modified.csv")

In [3]:
selected_data = pd.DataFrame()
selected_data['name'] = data['name']
selected_data['tags'] = data['tags']
selected_data['base_model'] = data['base_model']

In [4]:
selected_data["base_model"]

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
5902                                                  NaN
5903                                                  NaN
5904    ['yunconglong/Truthful_DPO_TomGrc_FusionNet_7B...
5905                                                  NaN
5906                                                   []
Name: base_model, Length: 5907, dtype: object

In [5]:
def insert_to_map(datamap, dataset, model):
    try:
        values = datamap[dataset]
    except KeyError:
        values = datamap[dataset] = []

    values.append(model)

In [11]:
datasets = []
datasets_map = {}

for index, row in selected_data.iterrows():
    tag_string = str(row["tags"])
    dataset_string = str(row["base_model"])

    if tag_string != "nan":
        tag_list = eval(tag_string)
        for tag in tag_list:
            if "base_model:" in tag:
                tag = tag.removeprefix("base_model:")
                # print(tag)
                datasets.append(tag)
                insert_to_map(datasets_map, tag, row['name'])
                
    
    if dataset_string != "nan" and dataset_string != "unknown":
        if dataset_string.startswith("["):
            dataset_list = eval(dataset_string)
        else:
            dataset_list = [dataset_string]
        if isinstance(dataset_list, (list, tuple, np.ndarray)):
            for tag in dataset_list:
                # print(tag)
                datasets.append(tag)
                insert_to_map(datasets_map, tag, row['name'])

In [12]:
print(f"Len of base_models before dedup: {len(datasets)}")
datasets = set(datasets)
print(f"Len of base_models after dedup: {len(datasets)}")

Len of base_models before dedup: 4777
Len of base_models after dedup: 983


In [13]:
known_trained_models = set(model for models in datasets_map.values() for model in models)
df = pd.DataFrame(columns=datasets_map.keys(), index=known_trained_models).fillna(0)


for dataset, models in datasets_map.items():
    for model in models:
        df.at[model, dataset] = 1

# Export the DataFrame to CSV
df.to_csv('model_base_model_matrix.csv')

print(df)

                                            moreh/MoMo-72B-LoRA-V1.4  \
Novocoders/Mistral-NeuralDPO-v0.7                                  0   
openchat/openchat-3.5-0106                                         0   
fhai50032/RolePlayLake-7B                                          0   
allenai/tulu-2-dpo-70b                                             0   
automerger/YamShadow-7B                                            0   
...                                                              ...   
mahiatlinux/ShadowDolph-7B-v1                                      0   
Gille/StrangeMerges_22-7B-slerp                                    0   
Undi95/Miqu-MS-70B                                                 0   
tianlinliu0121/zephyr-7b-dpo-full-beta-0.2                         0   
wandb/gemma-7b-zephyr-dpo                                          0   

                                            moreh/MoMo-72B-lora-1.8.7-DPO  \
Novocoders/Mistral-NeuralDPO-v0.7                         